In [1]:
import pandas as pd 
import json 
import requests
import os 
from dotenv import load_dotenv
import sys 

def print_json(data:json):
    """json 형태의 데이터를 출력하는 함수입니다.

    Args:
        data (json): json 형태의 데이터
    """
    pretty_json = json.dumps(data, indent=4)
    print(pretty_json)

In [2]:
data = pd.read_csv("../data//추천장소통합리스트.csv")

In [3]:
data.head()

,목적지명,분류,지역,위도,경도,방문건수순위,구글지도평점,데이터랩점수,방문건수순위점수,최종점수
0,솥뚜껑매운탕,식당,공주,36.479281,127.047294,1,0.88,0.64,1.00,2.52
1,곰골식당,식당,공주,36.451845,127.120543,2,0.83,0.64,0.95,2.42
2,베이커리밤마을,식당,공주,36.464752,127.122985,3,0.85,0.64,0.89,2.38
3,공주쌍신집칼국수,식당,공주,36.481470,127.119963,4,0.87,0.64,0.84,2.35
4,피탕김탕,식당,공주,36.477979,127.139030,5,0.00,0.64,0.79,1.43
